# Compute oxygen partial pressure

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
from itertools import product

import data_collections as dc
import funnel
import intake
import numpy as np
import util
import xarray as xr
import yaml

/glade/work/mclong/miniconda3/envs/metabolic/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
try:
    cluster
    client
except:
    cluster, client = util.get_ClusterClient(memory='100GB')
    cluster.scale(34)
client

/glade/work/mclong/miniconda3/envs/metabolic/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46615 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/46615/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/46615/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.15:42196,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/46615/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
sub_spec = dict(
    name='drift-corrected',
    experiment=['20C', 'RCP85'],
    member_id=dc.ocean_bgc_member_ids[1:],  # first time level for ensemble member 1 different
)

catalog = funnel.to_intake_esm(agg_member_id=False).search(**sub_spec)
catalog.df

,experiment,component,stream,member_id,variable,name,path
0,20C,ocn,pop.h,26,O2_PRODUCTION,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
1,20C,ocn,pop.h,32,SALT,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
2,20C,ocn,pop.h,30,O2_CONSUMPTION,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
3,20C,ocn,pop.h,10,O2_PRODUCTION,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
4,20C,ocn,pop.h,30,TEMP,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
...,...,...,...,...,...,...,...
445,RCP85,ocn,pop.h,104,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
446,RCP85,ocn,pop.h,104,vol_habitat_trait_spc,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
447,RCP85,ocn,pop.h,105,depth_habitat_trait_wgt,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
448,RCP85,ocn,pop.h,105,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...


In [5]:
catalog.search(experiment='RCP85', member_id=10).df

,experiment,component,stream,member_id,variable,name,path
0,RCP85,ocn,pop.h,10,TEMP,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
1,RCP85,ocn,pop.h,10,O2_CONSUMPTION,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
2,RCP85,ocn,pop.h,10,SALT,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
3,RCP85,ocn,pop.h,10,O2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
4,RCP85,ocn,pop.h,10,depth_habitat_trait_wgt,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
5,RCP85,ocn,pop.h,10,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
6,RCP85,ocn,pop.h,10,vol_habitat_trait_spc,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...


In [6]:
clobber = False

experiment_list = catalog.unique('experiment')['experiment']['values']
member_id_list = catalog.unique('member_id')['member_id']['values']

stream = 'pop.h'
component = 'ocn'

# refresh catalog
catalog = funnel.to_intake_esm(agg_member_id=False).search(**sub_spec)

for experiment, member_id in product(experiment_list, member_id_list):
    cat = catalog.search(
        experiment=experiment,
        member_id=member_id,
        stream=stream,
        component=component,
        variable=['TEMP', 'SALT', 'O2'],
    )

    # ensure variables
    missing_vars = {'TEMP', 'SALT', 'O2'} - set(cat.df.variable.to_list())
    if missing_vars:
        print(f'missing vars for {experiment}.{member_id:03d}: {missing_vars}')
        continue

    # check for existing cache file
    variable = 'pO2'
    asset = dc.fnl_gen_cache_file_name(experiment, component, stream, member_id, variable)

    if clobber and os.path.exists(asset):
        print(f'removing: {asset}')
        shutil.rmtree(asset)

    if os.path.exists(asset):
        print(f'exists: {asset}')
        continue

    # compute pO2
    print(f'computing/writing: {asset}')
    dset = cat.to_dataset_dict()
    assert len(dset.keys()) == 1
    _, ds = dset.popitem()

    ds = dc.compute_pO2(ds)
    ds.to_zarr(asset, mode='w', consolidated=True)

    dc.fnl_make_cache(experiment, component, stream, member_id, variable)

exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.009.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.010.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.011.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.012.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.013.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.014.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.015.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.RCP85.ocn.pop.h.016.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-meta

In [7]:
client.close()
cluster.close()